## Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import random
import ssl
import imageio
from IPython import display
from urllib import request
import re
import tempfile
from keras import backend as K
import sys
import csv
from collections import deque
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from google.colab import drive
import gc

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np # MATRIX OPERATIONS
import pandas as pd # EFFICIENT DATA STRUCTURES
import matplotlib.pyplot as plt # GRAPHING AND VISUALIZATIONS
%matplotlib inline
import math # MATHEMATICAL OPERATIONS
import cv2 # IMAGE PROCESSING - OPENCV
import os
from glob import glob # FILE OPERATIONS
import itertools

# KERAS AND SKLEARN MODULES
import keras
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Flatten,MaxPool2D
from keras.layers import Dense, Activation, Dropout, Flatten,  BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,CSVLogger, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
seed_constant = 30
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

## Mount drive

In [ ]:
#importing google drive in google colab
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DATASET_DIR = '/content/gdrive/MyDrive/sign_videos_sample_a_6'
FOLDER_ROOT = '/content/'

In [ ]:
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
SEQUENCE_LENGTH = 20

CLASSES_LIST = ['call_the_ambulance','i_am_a_student','i_can_not_speak', 'how_are_you', "i_don't_understand", 'extra_class'] # 5 a, 

# CLASSES_LIST = ['i_need_your_help', 'thank_you', 'what_is_the_time', 'what_is_your_name', 'yes'] #5 b


In [ ]:
#creating the Matplotlib figure and specify the size of the figure
plt.figure(figsize=(20,20))

with open(FOLDER_ROOT+'dataset.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  for c in CLASSES_LIST:
    path = os.path.join(DATASET_DIR, c+"/")
    for i in os.listdir(path):
      writer.writerow([CLASSES_LIST.index(c), os.path.join(path, i)])


train_df = pd.read_csv(FOLDER_ROOT+'dataset.csv', header=None)
train_df.columns = ["class", "path"]
train_df = train_df.astype({"class": str})
train_df.head()

,class,path
0,0,/content/gdrive/MyDrive/sign_videos_sample_a_6...
1,0,/content/gdrive/MyDrive/sign_videos_sample_a_6...
2,0,/content/gdrive/MyDrive/sign_videos_sample_a_6...
3,0,/content/gdrive/MyDrive/sign_videos_sample_a_6...
4,0,/content/gdrive/MyDrive/sign_videos_sample_a_6...


<Figure size 1440x1440 with 0 Axes>

In [ ]:
label_processor = tf.keras.layers.experimental.preprocessing.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["class"])
)
print(label_processor.get_vocabulary())
class_vocab = label_processor.get_vocabulary()

['0', '1', '2', '3', '4', '5']


In [ ]:
def frames_extraction(video_path):

  frames_list= []

  video_reader = cv2.VideoCapture(video_path)

  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

  for frame_counter in range(SEQUENCE_LENGTH):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

    success, frame = video_reader.read()

    if not success:
      break
      
    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

    normalized_frame = resized_frame /255

    frames_list.append(normalized_frame)

  video_reader.release()
  return frames_list

In [ ]:
def create_dataset():
  features = []
  labels = []
  video_files_paths = []

  for class_index, class_name in enumerate(CLASSES_LIST):
    print(f'Extracting Data of Class: {class_name}')

    files_list = os.listdir(os.path.join(DATASET_DIR,class_name))

    for file_name in files_list:
      video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
      frames = frames_extraction(video_file_path)

      if len(frames) == SEQUENCE_LENGTH:
        features.append(frames)
        labels.append(class_index)
        video_files_paths.append(video_file_path)
        
  features = np.asarray(features)
  labels = np.array(labels)
  return features, labels, video_files_paths


In [ ]:
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: call_the_ambulance
Extracting Data of Class: i_am_a_student
Extracting Data of Class: i_can_not_speak
Extracting Data of Class: how_are_you
Extracting Data of Class: i_don't_understand
Extracting Data of Class: extra_class


In [ ]:
print(features.shape)
print(labels.shape)

(451, 20, 224, 224, 3)
(451,)


In [ ]:
one_hot_encoded_labels = to_categorical(labels)
print(one_hot_encoded_labels.shape)

(451, 6)


In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                           test_size = 0.3, shuffle = True,
                                                                           random_state = seed_constant)



In [ ]:
gc.collect()

407

In [ ]:
def create_convlstm_model():
  model = Sequential()

  model.add(ConvLSTM2D(filters=2, kernel_size=(2,2), activation='relu', data_format="channels_last",
                     recurrent_dropout = 0.5, return_sequences =True, input_shape = (SEQUENCE_LENGTH,IMAGE_HEIGHT,IMAGE_WIDTH,3)))

  model.add(MaxPooling3D(pool_size=(1,2,2), padding='valid', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(0.5)))

  model.add(ConvLSTM2D(filters=4, kernel_size=(2,2), activation='relu', data_format="channels_last",
                     recurrent_dropout = 0.5, return_sequences =True))

  model.add(MaxPooling3D(pool_size=(1,2,2),padding='valid', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(0.5)))

  model.add(ConvLSTM2D(filters=16, kernel_size=(3,3), activation='tanh',data_format="channels_last",
                     recurrent_dropout = 0.4, return_sequences =True))

  model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(0.4)))

  model.add(Flatten())

  model.add(Dense(len(CLASSES_LIST), activation="softmax"))

  model.summary()
  return model

convlstm_model = create_convlstm_model()

print("Model created successfully!")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 20, 223, 223, 2)   168       
                                                                 
 max_pooling3d (MaxPooling3D  (None, 20, 111, 111, 2)  0         
 )                                                               
                                                                 
 time_distributed (TimeDistr  (None, 20, 111, 111, 2)  0         
 ibuted)                                                         
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 20, 110, 110, 4)   400       
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 20, 55, 55, 4)    0         
 3D)                                                             
                                                        

In [ ]:
# plot_model(convlstm_model, to_file='convlstm_model_structure_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("Features train:  ", features_train.shape)
print("Labels train: ", labels_train.shape)

Features train:   (315, 20, 224, 224, 3)
Labels train:  (315, 6)


In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, mode = 'min', restore_best_weights=True)
# filepath = "/tmp/video_classifier"
# checkpoint = keras.callbacks.ModelCheckpoint(
#     filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )


convlstm_model.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])

convlstm_model_training_history = convlstm_model.fit(x=features_train, y = labels_train, epochs = 50, batch_size=32,
                                                     validation_split=0.2,
                                                     callbacks=[early_stopping_callback])

Epoch 1/50


In [ ]:
#convlstm_model.load_weights(filepath)

In [ ]:
history = convlstm_model_training_history

print("Training accuracy: ", history.history['accuracy'][-1]*100)
print("Training Loss: ", history.history['loss'][-1])
print("-----------------------------------------")
print("Validation accuracy: ", history.history['val_accuracy'][-1]*100)
print("Validation Loss: ", history.history['val_loss'][-1])
print("-----------------------------------------\n")

accuracy = convlstm_model.evaluate(features_test, labels_test, verbose=0);
print(f"Test Accuracy: {round(accuracy[1] * 100, 2)}%")
print(f"Test Loss: {round(accuracy[0], 2)}%")


# Plots
plt.figure(1)

# summarize history for accuracy

plt.subplot(211)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')

# summarize history for loss

plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

plt.tight_layout()

plt.show()

In [ ]:
convlstm_model.save("convlstm_model.h5")

In [ ]:
convlstm_model = keras.models.load_model('convlstm_model.h5')

### **Single Video Inference**

In [ ]:
from tensorflow_docs.vis import embed

def load_video(video_path):
  cap = cv2.VideoCapture(video_path)
  frames = []

  video_reader = cv2.VideoCapture(video_path)

  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)

      if len(frames) == SEQUENCE_LENGTH:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0
  

def prepare_single_video(video_path):

    single_video_features = []

    frames = frames_extraction(video_path)
    
    if len(frames) == SEQUENCE_LENGTH:
      single_video_features.append(frames)
      
    features = np.asarray(single_video_features)
    
    return features

def sequence_prediction(video_path):
  
    class_vocab = label_processor.get_vocabulary() 

    frames = load_video(video_path)

    frame_features = prepare_single_video(video_path)
    probabilities = convlstm_model.predict(frame_features)

    v = ['call_the_ambulance','i_am_a_student','i_can_not_speak', 'how_are_you', "i_don't_understand", 'extra_class'] # 5 a, 5 b

    pred_class = v[np.argmax(probabilities)]
    print(pred_class)
    #for i in np.argsort(probabilities)[::-1]:  #Add [::-1][:no] for starting fix number samples
     # print(f"  {str(v[class_vocab[i].astype(int)])} :{class_vocab[i]}: {probabilities[i] * 100:5.2f}%")

    frames_data = np.array(frames, dtype=np.float32)
    return frames_data

def to_gif(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, fps=30)
    return embed.embed_file('./animation.gif')

In [ ]:
y_pred = convlstm_model.predict(features_test) 
y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(labels_test, axis=1)
cm = confusion_matrix(Y_test, y_pred)
print(cm)

In [ ]:
from sklearn.metrics import precision_score
precision_score(Y_test,y_pred, average='macro')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(Y_test,y_pred, average='macro')

### **Single Video Predictions**

In [ ]:
test_video = np.random.choice(train_df["path"].values.tolist())  # Videos from train data. Note: We dont have test dataframe

#test_video = '/content/sign_videos_sample_a_10/how_are_you/how_are_you-11.mp4'
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)

#to_gif(test_frames[:SEQUENCE_LENGTH])